<a href="https://colab.research.google.com/github/leafy-dandelion/mag-repo/blob/main/RL/HW1_Liubyma_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HW1 programming part

(a) (coding) Read through vi_and_pi.py and implement policy_evaluation, policy_improvement
and policy_iteration. The stopping tolerance (defined as maxs |Vold(s) − Vnew(s)|) is tol =
10−3
. Use γ = 0.9. Return the optimal value function and the optimal policy. [10pts]

(b) (coding) Implement value_iteration in vi_and_pi.py. The stopping tolerance is tol =
10−3
. Use γ = 0.9. Return the optimal value function and the optimal policy. [10 pts]

(c) (written) Run both methods on the Deterministic-4x4-FrozenLake-v0 and
Stochastic-4x4-FrozenLake-v0 environments. In the second environment, the dynamics of the
world are stochastic. How does stochasticity affect the number of iterations required, and the
resulting policy? [5 pts]

## important set up

In [ ]:
!pip install gym==0.10.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title lake_envs.py
# coding: utf-8
"""Defines some frozen lake maps."""
import gym
from gym.envs.toy_text import frozen_lake, discrete
from gym.envs.registration import register

# De-register environments if there is a collision
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
	if 'Deterministic-4x4-FrozenLake-v0' in env:
		del gym.envs.registration.registry.env_specs[env]

	elif 'Deterministic-8x8-FrozenLake-v0' in env:
		del gym.envs.registration.registry.env_specs[env]

	elif 'Stochastic-4x4-FrozenLake-v0' in env:
		del gym.envs.registration.registry.env_specs[env]


register(
    id='Deterministic-4x4-FrozenLake-v0',
    entry_point='gym.envs.toy_text.frozen_lake:FrozenLakeEnv',
    kwargs={'map_name': '4x4',
            'is_slippery': False})

register(
    id='Deterministic-8x8-FrozenLake-v0',
    entry_point='gym.envs.toy_text.frozen_lake:FrozenLakeEnv',
    kwargs={'map_name': '8x8',
            'is_slippery': False})

register(
    id='Stochastic-4x4-FrozenLake-v0',
    entry_point='gym.envs.toy_text.frozen_lake:FrozenLakeEnv',
    kwargs={'map_name': '4x4',
            'is_slippery': True})

## task (a) and (b)

copied from vi_and_pi.py

In [ ]:
import argparse
import numpy as np
import gym
import time
#next line is not needed as we copied the program
# from lake_envs import *

In [ ]:
### MDP Value Iteration and Policy Iteration
import argparse
import numpy as np
import gym
import time
#from lake_envs import *

np.set_printoptions(precision=3)

parser = argparse.ArgumentParser(description='A program to run assignment 1 implementations.',
								formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument("--env", 
					help="The name of the environment to run your algorithm on.", 
					choices=["Deterministic-4x4-FrozenLake-v0","Stochastic-4x4-FrozenLake-v0"],
					default="Deterministic-4x4-FrozenLake-v0")

"""
For policy_evaluation, policy_improvement, policy_iteration and value_iteration,
the parameters P, nS, nA, gamma are defined as follows:

	P: nested dictionary
		From gym.core.Environment
		For each pair of states in [1, nS] and actions in [1, nA], P[state][action] is a
		tuple of the form (probability, nextstate, reward, terminal) where
			- probability: float
				the probability of transitioning from "state" to "nextstate" with "action"
			- nextstate: int
				denotes the state we transition to (in range [0, nS - 1])
			- reward: int
				either 0 or 1, the reward for transitioning from "state" to
				"nextstate" with "action"
			- terminal: bool
			  True when "nextstate" is a terminal state (hole or goal), False otherwise
	nS: int
		number of states in the environment
	nA: int
		number of actions in the environment
	gamma: float
		Discount factor. Number in range [0, 1)
"""

def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3):
  """Evaluate the value function from a given policy.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	policy: np.array[nS]
		The policy to evaluate. Maps states to actions.
	tol: float
		Terminate policy evaluation when
			max |value_function(s) - prev_value_function(s)| < tol
	Returns
	-------
	value_function: np.ndarray[nS]
		The value function of the given policy, where value_function[s] is
		the value of state s
    """
  iter = 3000 #set maximum of iteration
  value_function = np.zeros(nS)
  
 
  i = 1
  d = tol+1
  while d > tol and i <= iter:
    value_function_new = np.zeros(nS)
    for s in range(nS):
      # for a in range(nA):
				# P[s][a][0] - transition probability
				# P[s][a][1] - next state
				# P[s][a][2] - reward
				# P[s][a][3] - 1 if next state is target 0 otherwise
      a = policy[s]
      for p, s_, r, t in P[s][a]:
        # if t: #if target                   
        #   value_function_new[s] += policy[s]*p*r #no next 
        # else:
        if not t:
          value_function_new[s] += r+gamma*p*value_function[int(s_)] #plus next state
        else:
          value_function_new[s] += r
    d = max(abs(value_function-value_function_new))
    i+=1
    value_function = value_function_new
  return value_function


def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=0.9):
  """Given the value function from policy improve the policy.
  Parameters
  ----------
  P, nS, nA, gamma:
    defined at beginning of file
  value_from_policy: np.ndarray
    The value calculated from the policy
  policy: np.array
    The previous policy.
  Returns
  -------
  new_policy: np.ndarray[nS]
    An array of integers. Each integer is the optimal action to take
    in that state according to the environment dynamics and the
    given value function.
  """

  new_policy = np.zeros(nS, dtype='int')
 
  for s in range(nS):
    q_a = np.zeros(nA)
    for a in range(nA):
      for p, s_, r, t in P[s][a]:
        if not t:
          q_a[a] += r+gamma*p*value_from_policy[int(s_)]
        else:
          q_a[a]+=r
    new_policy[s] = np.argmax(q_a)
  return new_policy


def policy_iteration(P, nS, nA, gamma=0.9, tol=10e-3):
  """Runs policy iteration.

	You should call the policy_evaluation() and policy_improvement() methods to
	implement this method.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		tol parameter used in policy_evaluation()
	Returns:
	----------
	value_function: np.ndarray[nS]
	policy: np.ndarray[nS]
  """

  value_function = np.zeros(nS)
  #policy = np.zeros(nS, dtype=int)
  i = 0
  d = 1
  policy = np.random.randint(nA, size=nS)
  while i==0 or d > 0 :
    V = policy_evaluation(P, nS, nA, policy, gamma, tol)
    policy_improve = policy_improvement(P, nS, nA, V, policy, gamma)
    i+=1
    d = np.linalg.norm(policy - policy_improve, ord=1)
    policy = policy_improve 
  return value_function, policy

def value_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
  """
	Learn value function and policy by using value iteration method for a given
	gamma and environment.

	Parameters:
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		Terminate value iteration when
			max |value_function(s) - prev_value_function(s)| < tol
	Returns:
	----------
	value_function: np.ndarray[nS]
	policy: np.ndarray[nS]
  """

  value_function = np.zeros(nS)
  policy = np.zeros(nS, dtype=int)
  d = tol+1
  while d>tol:
    value_function_new = np.zeros(nS)
    for s in range(nS):
      op_a = np.zeros(nA)
      for a in range(nA): 
         for p, s_, r, t in P[s][a]:
           if not t:
             op_a[a] += r+gamma*p*value_function[int(s_)]
           else:
             op_a[a] += r
      value_function_new[s] = np.max(op_a)
      policy[s] = np.argmax(op_a)
    d=max(abs(value_function_new-value_function))
    value_function = value_function_new
  return value_function, policy

def render_single(env, policy, max_steps=100):
  """
    This function does not need to be modified
    Renders policy once on environment. Watch your agent play!

    Parameters
    ----------
    env: gym.core.Environment
      Environment to play on. Must have nS, nA, and P as
      attributes.
    Policy: np.array of shape [env.nS]
      The action to take at a given state
  """

  episode_reward = 0
  ob = env.reset()
  for t in range(max_steps):
    env.render()
    time.sleep(0.25)
    a = policy[ob]
    ob, rew, done, _ = env.step(a)
    episode_reward += rew
    if done:
      break
  env.render();
  if not done:
    print("The agent didn't reach a terminal state in {} steps.".format(max_steps))
  else:
  	print("Episode reward: %f" % episode_reward)

##Task (c): run for 2 different environments

### Deterministic

In [ ]:
# Edit below to run policy and value iteration on different environments and
# visualize the resulting policies in action!
# You may change the parameters in the functions below
#args = parser.parse_args("Deterministic-4x4-FrozenLake-v0")
	
	# Make gym environment
env = gym.make("Deterministic-4x4-FrozenLake-v0")

print("\n" + "-"*25 + "\nBeginning Policy Iteration\n" + "-"*25)

V_pi, p_pi = policy_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)
render_single(env, p_pi, 100)
print("\n" + "-"*25 + "\nBeginning Value Iteration\n" + "-"*25)

V_vi, p_vi = value_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)
render_single(env, p_vi, 100)

/usr/local/lib/python3.9/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)



-------------------------
Beginning Policy Iteration
-------------------------

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode reward: 1.000000

-------------------------
Beginning Value Iteration
-------------------------

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode reward: 1.000000


### Stochastic

In [ ]:
env = gym.make("Stochastic-4x4-FrozenLake-v0")

print("\n" + "-"*25 + "\nBeginning Policy Iteration\n" + "-"*25)

V_pi, p_pi = policy_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)
render_single(env, p_pi, 150)
print("\n" + "-"*25 + "\nBeginning Value Iteration\n" + "-"*25)

V_vi, p_vi = value_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)
render_single(env, p_vi, 150)

/usr/local/lib/python3.9/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)



-------------------------
Beginning Policy Iteration
-------------------------

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
S